In [2]:
cd "/UNIVERSIDAD/OneDrive - Universidad de Oviedo"

C:\UNIVERSIDAD\OneDrive - Universidad de Oviedo


In [3]:
cd "TESIS/DATOS/RealOviedo/Oviedo Moderno CF_JORNADA 12 - INTERRIAS_RawData"

C:\UNIVERSIDAD\OneDrive - Universidad de Oviedo\TESIS\DATOS\RealOviedo\Oviedo Moderno CF_JORNADA 12 - INTERRIAS_RawData


In [4]:
import pandas as pd
#Set format
pd.set_option('display.float_format', '{:.10f}'.format)

df = pd.read_excel("todas.xlsx")
#Filtering by hour
from datetime import time
df["local_time"] = pd.to_datetime(df.local_time)
filtro = (df.local_time.dt.time >= time(12, 2, 12)) & (df.local_time.dt.time < time(12, 49, 50))
df=df.loc[filtro, ]

In [80]:
#filter by player id=5 to obtain her heat map
filtroJugadora = df["DEV"]==10
df4=df.loc[filtroJugadora, ]

dfx=df4[['DEV','lat','lon']]
dfx = dfx.groupby(['DEV', 'lat', 'lon']).size().reset_index(name='frecuencia')

import plotly.express as px
px.set_mapbox_access_token('pk.eyJ1IjoibG9saXZlaXJhciIsImEiOiJjbHFmZ2I4cXcwd2p6Mm9tazI5Z3RybjBwIn0.5QzzXVPTSVGSaF7v7buy2A')
fig = px.density_mapbox(dfx, lat = 'lat', lon = 'lon', z = 'frecuencia',
                        radius = 5,
                        zoom = 17,
                        mapbox_style = 'satellite',
                        color_continuous_scale = 'rainbow',
                        opacity=1
                        )
fig.update_layout(
    mapbox=dict(
        bearing=0       
    ),
    uirevision=True,
)
fig.update(layout_coloraxis_showscale=False)

fig.show()


In [81]:
#To eliminate situations in which player stays
df4['distance'] = getDistanceBetweenPointsNew(df4['lat'] , df4['lon'] , df4['lat'].shift(1) , df4['lon'].shift(1),unit='meters')
#To eliminate situations in which player stays
filtroDistance = df4["distance"]>0.20
df4=df4.loc[filtroDistance, ]

C:\Users\Luis\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning:

invalid value encountered in arccos

C:\Users\Luis\AppData\Local\Temp\ipykernel_23744\3907812442.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [69]:
df4

,local_time,DEV,fecha,hora,lat,lon,vel,distance
899322,2023-11-26 12:02:24.000,10,2023-11-26,12:02:24,43.3553913333,-5.9229188333,7.5980000000,0.2300000000
899324,2023-11-26 12:02:24.200,10,2023-11-26,12:02:24,43.3553886667,-5.9229151667,7.8250000000,0.2300000000
899325,2023-11-26 12:02:24.300,10,2023-11-26,12:02:24,43.3553863333,-5.9229145000,8.8190000000,0.2700000000
899326,2023-11-26 12:02:24.400,10,2023-11-26,12:02:24,43.3553845000,-5.9229126667,8.7020000000,0.2700000000
899327,2023-11-26 12:02:24.500,10,2023-11-26,12:02:24,43.3553828333,-5.9229105000,8.8710000000,0.2500000000
...,...,...,...,...,...,...,...,...
927753,2023-11-26 12:49:47.200,10,2023-11-26,12:49:47,43.3553521667,-5.9232003333,7.9010000000,0.2300000000
927754,2023-11-26 12:49:47.300,10,2023-11-26,12:49:47,43.3553533333,-5.9231983333,7.3630000000,0.2100000000
927777,2023-11-26 12:49:49.600,10,2023-11-26,12:49:49,43.3553691667,-5.9231700000,6.7380000000,0.2300000000
927779,2023-11-26 12:49:49.800,10,2023-11-26,12:49:49,43.3553700000,-5.9231650000,8.3210000000,0.2300000000


In [82]:
import pandas as pd
import numpy as np
import plotly.express as px

# Supongamos que tienes un DataFrame llamado df con las columnas 'lat' y 'lon'

# Agrupar por latitud y longitud y contar cuántas veces aparece cada combinación
df_grouped = df4.groupby(['lat', 'lon']).size().reset_index(name='frecuencia')

# Ordenar el DataFrame por frecuencia en orden descendente
df_sorted = df_grouped.sort_values(by='frecuencia', ascending=False)

# Seleccionar las cuatro zonas con más frecuencia
top_zones = df_sorted.head(4)

# Calcular el centroide de las zonas seleccionadas
centroid_lat = np.mean(top_zones['lat'])
centroid_lon = np.mean(top_zones['lon'])

# Añadir una nueva fila al DataFrame con las coordenadas del centroide
df_centroid = pd.DataFrame({'lat': [centroid_lat], 'lon': [centroid_lon]})

# Concatenar el DataFrame original con el DataFrame del centroide
df_with_centroid = pd.concat([df_grouped, df_centroid])

# Crear el mapa de calor
fig = px.density_mapbox(df_with_centroid, lat='lat', lon='lon', z='frecuencia',
                        radius=5, zoom=18,
                        mapbox_style='satellite',
                        color_continuous_scale='Viridis')

# Añadir el punto del centroide al mapa de calor
fig.add_scattermapbox(lat=[centroid_lat], lon=[centroid_lon], mode='markers', marker=dict(size=10, color='yellow'))

# Mostrar el mapa
fig.show()



In [79]:
df4

,local_time,DEV,fecha,hora,lat,lon,vel,distance
899322,2023-11-26 12:02:24.000,10,2023-11-26,12:02:24,43.3553913333,-5.9229188333,7.5980000000,0.2300000000
899324,2023-11-26 12:02:24.200,10,2023-11-26,12:02:24,43.3553886667,-5.9229151667,7.8250000000,0.2300000000
899325,2023-11-26 12:02:24.300,10,2023-11-26,12:02:24,43.3553863333,-5.9229145000,8.8190000000,0.2700000000
899326,2023-11-26 12:02:24.400,10,2023-11-26,12:02:24,43.3553845000,-5.9229126667,8.7020000000,0.2700000000
899327,2023-11-26 12:02:24.500,10,2023-11-26,12:02:24,43.3553828333,-5.9229105000,8.8710000000,0.2500000000
...,...,...,...,...,...,...,...,...
927753,2023-11-26 12:49:47.200,10,2023-11-26,12:49:47,43.3553521667,-5.9232003333,7.9010000000,0.2300000000
927754,2023-11-26 12:49:47.300,10,2023-11-26,12:49:47,43.3553533333,-5.9231983333,7.3630000000,0.2100000000
927777,2023-11-26 12:49:49.600,10,2023-11-26,12:49:49,43.3553691667,-5.9231700000,6.7380000000,0.2300000000
927779,2023-11-26 12:49:49.800,10,2023-11-26,12:49:49,43.3553700000,-5.9231650000,8.3210000000,0.2300000000


In [83]:
df_centroid

,lat,lon
0,43.3553719583,-5.9227041250
